In [1]:
import numpy as np
import ot
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import accuracy_score
from scipy.io import savemat
import glob
import os
from sklearn.metrics import classification_report
#import cv2,os
from sklearn.cluster import SpectralClustering
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
import warnings
import scipy as sp
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
import sys
sys.path.append('../')
from HOTDA import *
warnings.filterwarnings("ignore")
%matplotlib notebook
import time

In [2]:
print('training..')
print('Time over 10 runs')
Score_mean=[]
domain=[]
for file in glob.glob(os.path.join('data/', '', '*')):
    data    = sp.io.loadmat(file)
    Xsource = data['Xsource'].astype(float)
    Ysource = data['Ysource'].astype(float)
    XTarget = data['XTarget'].astype(float)
    Xtest   = data['Xtest'].astype(float)
    Ytest   = data['Ytest'].astype(float)
    Score=[]
    t=time.time()
    for i in range(10):
        clustering  = SpectralClustering(n_clusters=10, eigen_solver='arpack', assign_labels='kmeans',affinity="nearest_neighbors",n_neighbors=7).fit(XTarget)
        S,a,mu,y_Transported_S=Source_target_processing(Xsource,Ysource[0])
        T,b,nu,yxss=Source_target_processing(XTarget,clustering.labels_)
        HOT,W=Hot(S,a,mu,T,b,nu,100,100)
        X_Transported_S=Mapping(S,T,a,b,HOT,17)
        classifier =  KNeighborsClassifier(n_neighbors=1)
        classifier.fit(X_Transported_S,y_Transported_S)
        Score.append(classifier.score(Xtest,Ytest[0]))
    t2=time.time()
    print(t2-t,''+file.split('_')[0][5]+'->'+file.split('_')[1][0])
    Score_mean.append(np.array(Score).mean())
    domain.append(''+file.split('_')[0][5]+'->'+file.split('_')[1][0])    

training..
Time over 10 runs
0.48443174362182617 D->A
0.5972700119018555 D->W
0.46039724349975586 D->C
0.4939751625061035 C->D
0.4536869525909424 A->D
0.48164916038513184 A->W
0.5141661167144775 A->C
0.59163498878479 C->A
0.5403618812561035 C->W
0.5433509349822998 W->A
0.609123945236206 W->C
0.6332528591156006 W->D


In [3]:
df = pd.DataFrame({
     'Domain': domain,
     'Score': Score_mean})
df.to_csv("Result.csv", sep=';', index=False)
df.sort_values('Domain')

,Domain,Score
6,A->C,0.800000
4,A->D,0.925234
5,A->W,0.967347
7,C->A,0.921806
3,C->D,0.962617
8,C->W,0.951020
0,D->A,0.912996
2,D->C,0.784716
1,D->W,0.963265
9,W->A,0.918502
